<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-28 14:26:16
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  5.37 L
Current:  1.32 C
-------------------
Today PnL: 30.73 K (0.23%)
Current PnL: -16.30 L (-11.36%)
CY Booked + Current PnL: -11.84 L (-8.25%)
-------------------
Total profit:  3.79 L
Total loss:  -20.09 L
-------------------
Total Booked + Current PnL: 15.14 L (12.99%)
Total Booked PnL: 31.44 L (26.98%)
Curr Year Booked PnL: 4.46 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 79.05 L (59.84%)
-------------------
Est LTT:  2.72 C
Est LTT PnL: 1.40 C (106.19%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.00,-5.71,H-LC,88.17,232041.0,37349.0,11741.0,7.39,19.18,5.06,25.21,29.0,3.18,1.83,28.31,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.00,3.85,X-LC,47.31,154355.0,13546.0,11978.0,-0.42,9.62,7.76,18.13,17.0,1.13,1.22,21.88,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100337.0,21975.0,12061.0,0.64,28.04,12.02,43.43,71.0,1.82,0.79,34.20,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,7.81,H-SC,84.95,158933.0,21073.0,16688.0,-0.55,15.29,10.50,27.39,118.0,1.26,1.25,37.75,X40,ATH,FMCG
56,PIDILITIND,2504.06,3576.00,-18.44,X-LC,19.35,90327.0,15205.0,16954.0,-0.55,20.24,18.77,42.81,11.0,0.90,0.71,13.28,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-5.71,H-LC,88.17,232041.0,37349.0,11741.0,7.39,19.18,5.06,25.21,29.0,3.18,1.83,28.31,XY25,NTT,INSURANCE
16,CAMPUS,294.86,393.0,NaN,NaN,95.70,165465.0,-2310.0,58144.0,5.71,-1.38,35.14,33.28,175.0,-0.04,1.31,28.11,XY24,NTT,FOOTWEAR
75,SURYODAY,189.78,240.0,55.70,H-SC,69.89,110600.0,-39326.0,79002.0,5.60,-26.23,71.43,26.46,129.0,-0.50,0.87,41.69,XR,NTT,BANKS
87,VALIANTORG,512.64,838.0,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,64.52,64608.0,-48941.0,88991.0,3.16,-43.10,137.74,35.27,238.0,-0.55,0.51,58.13,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,1149.24,-11.10,X-SC,17.20,81240.0,-19537.0,69314.0,-3.99,-19.39,85.32,49.39,65.0,-0.28,0.64,5.72,X40N,ATH,DURABLES
81,TITAGARH,1097.23,1548.00,-9.25,H-SC,90.32,159434.0,-32581.0,111460.0,-2.98,-16.97,69.91,41.08,127.0,-0.29,1.26,34.94,XY24,NTT,ENGINEERING
58,QUESS,711.66,986.00,NaN,NaN,7.53,70836.0,-85018.0,145100.0,-2.30,-54.55,204.84,38.55,173.0,-0.59,0.56,8.47,XY24,NTT,MISC
69,SFL,1016.35,1287.00,260.91,H-SC,6.45,162798.0,-99420.0,169245.0,-1.98,-37.92,103.96,26.63,136.0,-0.59,1.28,0.00,XY24,NTT,MISC
42,IRCTC,904.87,1269.00,-26.41,M-MC,39.78,121290.0,-19870.0,76680.0,-1.97,-14.08,63.22,40.24,150.0,-0.26,0.96,15.88,XY24,BTT,TRAVEL


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.06,X-LC,55.91,250584.0,19985.0,111134.0,0.38,8.67,44.35,56.86,1.0,0.18,1.98,14.37,X40,BTT,IT
80,TCS,3841.06,4998.00,-16.16,X-LC,38.71,241638.0,-23395.0,103228.0,0.09,-8.83,42.72,30.12,2.0,-0.23,1.91,8.37,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.24,X-LC,83.87,184158.0,-16152.0,74952.0,0.15,-8.06,40.70,29.35,4.0,-0.22,1.45,27.23,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-7.94,X-LC,82.80,83330.0,-26842.0,115920.0,1.03,-24.36,139.11,80.85,5.0,-0.23,0.66,22.73,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-6.75,X-LC,8.60,210052.0,-6687.0,71355.0,-1.80,-3.09,33.97,29.84,6.0,-0.09,1.66,10.19,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,9.27,H-SC,100.00,126362.0,28716.0,18954.0,1.41,29.41,15.00,48.82,138.0,1.52,1.00,40.38,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,63.44,102697.0,17596.0,25510.0,1.29,20.68,24.84,50.65,92.0,0.69,0.81,29.09,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100337.0,21975.0,12061.0,0.64,28.04,12.02,43.43,71.0,1.82,0.79,34.20,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-18.44,X-LC,19.35,90327.0,15205.0,16954.0,-0.55,20.24,18.77,42.81,11.0,0.90,0.71,13.28,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.00,M-SC,65.59,145160.0,7582.0,86835.0,0.11,5.51,59.82,68.62,182.0,0.09,1.15,25.84,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,9.27,H-SC,100.00,126362.0,28716.0,18954.0,1.41,29.41,15.00,48.82,138.0,1.52,1.00,40.38,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,63.44,102697.0,17596.0,25510.0,1.29,20.68,24.84,50.65,92.0,0.69,0.81,29.09,X40N,ATH,CABLES
28,GILLETTE,8109.44,10330.69,7.81,H-SC,84.95,158933.0,21073.0,16688.0,-0.55,15.29,10.50,27.39,118.0,1.26,1.25,37.75,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,-0.04,M-LC,89.25,100337.0,21975.0,12061.0,0.64,28.04,12.02,43.43,71.0,1.82,0.79,34.20,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.67,H-SC,35.48,111642.0,9638.0,35391.0,0.27,9.45,31.70,44.14,111.0,0.27,0.88,27.22,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,23.66,134785.0,5633.0,81437.0,1.47,4.36,60.42,67.42,155.0,0.07,1.06,31.09,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,74.19,152131.0,1218.0,59240.0,0.19,0.81,38.94,40.06,89.0,0.02,1.20,24.43,AR,ATH,AUTO
92,WIPRO,243.46,420.00,-10.62,M-LC,43.01,153376.0,2431.0,107026.0,-0.35,1.61,69.78,72.51,69.0,0.02,1.21,15.48,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.96,H-SC,48.39,122716.0,-620.0,137589.0,0.35,-0.50,112.12,111.05,108.0,-0.00,0.97,20.24,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,64.52,64608.0,-48941.0,88991.0,3.16,-43.10,137.74,35.27,238.0,-0.55,0.51,58.13,XR,NTT,HOTELS
82,TRIDENT,37.35,48.0,68.21,M-SC,92.47,80818.0,-11436.0,37742.0,0.03,-12.40,46.70,28.51,220.0,-0.30,0.64,38.29,XR,NTT,TEXTILES
52,MASFIN,326.60,399.5,-23.22,H-SC,56.99,87345.0,-10635.0,32501.0,-0.50,-10.85,37.21,22.32,130.0,-0.33,0.69,26.68,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-0.04,M-LC,89.25,100337.0,21975.0,12061.0,0.64,28.04,12.02,43.43,71.0,1.82,0.79,34.20,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,9.27,H-SC,100.00,126362.0,28716.0,18954.0,1.41,29.41,15.00,48.82,138.0,1.52,1.00,40.38,X40,ATH,PHARMA
87,VALIANTORG,512.64,838.00,-367.00,H-SC,98.92,113464.0,-19822.0,104421.0,4.99,-14.87,92.03,63.47,131.0,-0.19,0.90,90.19,XR,NTT,CHEMICALS
77,TANLA,917.30,1963.11,-35.39,H-SC,97.85,158375.0,-81040.0,354000.0,1.86,-33.85,223.52,114.01,110.0,-0.23,1.25,45.25,AR,ATH,IT
7,ASIANTILES,75.41,137.00,6000.00,M-SC,96.77,66940.0,-26870.0,103489.0,1.07,-28.64,154.60,81.67,203.0,-0.26,0.53,29.63,XR,NTT,CERAMICS
16,CAMPUS,294.86,393.00,NaN,NaN,95.70,165465.0,-2310.0,58144.0,5.71,-1.38,35.14,33.28,175.0,-0.04,1.31,28.11,XY24,NTT,FOOTWEAR


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.51
1,25,40.55
2,50,69.86


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.49
XY25    14.49
X40     14.28
X40N    13.70
AR      11.86
XR      10.60
X200     1.45
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.68
M-SC    18.17
X-LC    17.40
H-MC     8.82
M-LC     7.82
H-LC     6.62
X-MC     5.70
M-MC     5.62
X-SC     2.82
L-SC     2.34
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.19,-0.59,32.01
IT,10.09,-17.13,81.14
BANKS,9.79,-12.90,64.80
MISC,6.52,-30.77,88.61
FINANCE,6.05,-24.75,83.99
PAINTS,5.49,-18.29,53.29
INSURANCE,5.45,3.40,25.56
ELECTRICAL,4.83,-4.71,56.84
AUTO,4.51,-9.31,57.06


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2856985.0,26
AR,1233653.0,12
X40N,1214280.0,17
XR,1109013.0,14
X40,750862.0,11
XY25,466175.0,10
SR,199046.0,2
X200,74952.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2269057.0,21
M-SC,2017997.0,21
X-LC,775674.0,12
M-MC,555977.0,5
H-MC,509898.0,8
X-SC,351901.0,4
M-LC,313044.0,6
X-MC,308857.0,5
L-SC,300198.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930044.0      6
M-SC       XY24      739435.0      6
H-SC       AR        550534.0      3
           X40N      501522.0      6
M-SC       X40N      469216.0      6
X-LC       X40       434986.0      5
M-MC       XY24      393937.0      3
M-SC       XR        289251.0      4
H-SC       XR        251315.0      4
H-LC       AR        245269.0      3
M-SC       X40       219879.0      2
M-LC       XY24      187656.0      4
H-MC       XY24      179087.0      2
L-SC       XR        152511.0      2
X-LC       AR        140958.0      2
X-MC       XY24      139592.0      2
X-LC       XY25      124778.0      4
M-SC       XY25      122302.0      1
X-SC       X40N      121580.0      2
           XR        115837.0      1
           SR        114484.0      1
H-MC       XR        111636.0      1
M-LC       XR        107026.0      1
X-MC       X40N       96452.0      2
M-SC       AR         93352.0      1
           SR         84562.0      1
L-SC       XY24       83990.0      1
M-MC       XR         81437.0      1
           AR         80603.0      1
X-LC       X200       74952.0      1
H-MC       XY25       74070.0      1
X-MC       XY25       72813.0      1
L-SC       AR         63697.0      1
H-MC       X40        60355.0      2
           AR         59240.0      1
L-LC       XY25       42109.0      1
H-SC       X40        35642.0      2
H-MC       X40N       25510.0      1
M-LC       XY25       18362.0      1
H-LC       XY25       11741.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
